In [1]:
import requests
import json
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from networkx import *
import random
import plotly.plotly as py
import plotly.graph_objs as go
#import plotly.graph_objs.scatter as go
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
api_key = 'f0f8be8d318201330fe3f845c883fa9f45ac014f'

In [3]:
import json
import requests

#setting up a caching file
CACHE_FNAME = 'cache_texts.json'

#setting up caching pattern
try:
    cache_file = open(CACHE_FNAME, 'r') # Try to read the data from the file
    cache_contents = cache_file.read()  # If it's there, get it into a string
    CACHE_DICTION = json.loads(cache_contents) # And then load it into a dictionary
    cache_file.close() # Close the file, we're good, we got the data in a dictionary.
except:
    CACHE_DICTION = {}

def get_items(page_number):
    baseurl = "https://digital.janeaddams.ramapo.edu/api/items"
    url_params = {"key" : api_key, "item_type" : "Person", "page" : page_number}
    page_number_full = "page_{}".format(str(page_number)) #this will be the key for each page of results in the cache diction
    if page_number_full in CACHE_DICTION:
        print("Data was in the cache")
        return CACHE_DICTION[page_number_full]
    else:
        print("making a request for new data")
        response = requests.get(baseurl, url_params)
        CACHE_DICTION[page_number_full] = response.json()  #try response.json???
        dumped_json_cache = json.dumps(CACHE_DICTION)
        fw = open(CACHE_FNAME,"w")
        fw.write(dumped_json_cache)
        fw.close()
    return CACHE_DICTION[page_number_full]

for i in range(1,258):
    try:
        get_items(i)
    except:
        print("Not working")

Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was in the cache
Data was i

In [4]:
with open("cache_texts.json") as json_file:
    data = json.load(json_file)

In [6]:
def pull_tags(data, tag_name):
    name_tag_dict = {}
    for page in data:
        for item in data[page]:
            tag_temp_list = []
            for element in item["element_texts"]:
                if element["element"]["name"] == "Title":
                    name = element["text"]
            for d in item['tags']:
                tag_temp_list.append(d['name'])
            if tag_name in tag_temp_list:
                name_tag_dict[name] = tag_temp_list
    return name_tag_dict


In [19]:
def geo_api_call(data, name_tag_dict):
    new_new_dict = {}
    geo_dict = {}
    for page in data:
        for item in data[page]:
            for element in item["element_texts"]:
                if element["element"]["name"] == "Title":
                    name = element["text"]
                try:
                    if name in name_tag_dict.keys():
                        new_new_dict[name] = item['extended_resources']['geolocations']['id']
                except:
                    pass

    for person in new_new_dict.keys():
        p_id = new_new_dict[person]
        api_url = 'https://digital.janeaddams.ramapo.edu/api/geolocations/'+str(p_id)
        r = requests.get(api_url)
        geo_dict[person] = r.json() 
        
    return geo_dict

In [23]:
def map_data(geo_dict):
    lat_list = []
    long_list = []
    person_list = []
    occupation_list = []

    for person in geo_dict.keys():
        try:
            lat_list.append(geo_dict[person]['latitude'])
            long_list.append(geo_dict[person]['longitude'])
            person_list.append(person)
                
        except:
            pass
    
    dict_to_df = {}
    dict_to_df['person'] = person_list
    dict_to_df['latitude'] = lat_list
    dict_to_df['longitude'] = long_list
    peace_geo_df = pd.DataFrame.from_dict(dict_to_df)
    data = [ go.Scattergeo(
        locationmode = 'ISO-3',
        lon = peace_geo_df['longitude'],
        lat = peace_geo_df['latitude'],
        text = peace_geo_df['person'],
        mode = 'markers') ]
    return data

In [16]:
# #pulling occupation
# occupation_list = []
# for page in data:
#     for item in data[page]:
#         for element in item["element_texts"]:
#             if element["element"]["name"] == "Occupation":
#                 occupation_list.append(element["text"])

In [24]:
def all_functions(data, tag_name):
    step1 = pull_tags(data, tag_name)
    step2 = geo_api_call(data, step1)
    step3 = map_data(step2)
    return step3

In [25]:
peace = all_functions(data, 'Peace')
py.iplot(peace, filename = 'Peace Movement Network Map')

In [26]:
child_labor = all_functions(data, 'Child Labor')
py.iplot(child_labor, filename = 'Child Labor Network Map')

In [27]:
child_welfare = all_functions(data, 'Child Welfare')
py.iplot(child_welfare, filename = 'Child Welfare Network Map')

In [38]:
# prog_party = all_functions(data, 'Progressive Party')
# py.iplot = (prog_party)

In [29]:
settlement_mov = all_functions(data, 'Settlement Movement')
py.iplot(settlement_mov, filename = 'Settlement Movement Map')

In [30]:
social_work = all_functions(data, 'Social Work')
py.iplot(social_work, filename = 'Social Work Network Map')

In [31]:
socialism = all_functions(data, 'Socialism')
py.iplot(socialism, filename = 'Socialism Network Map')

In [32]:
woman_suffrage = all_functions(data, 'Woman Suffrage')
py.iplot(woman_suffrage, filename = 'Woman Suffrage Network Map')

In [39]:
# womens_rights = all_functions(data, "Women's Rights")
# py.iplot(womens_rights, filename = 'Womens Rights Network Map')

In [41]:
#BELOW IS CODE FOR INITIAL SOCIAL NETWORK GRAPH CONCEPT - WILL NOT BE USED ON ACTUAL SITE

In [ ]:
#finding tags mentioned more than once
popular_tag_list = []
for item in tag_count_d.keys():
    if tag_count_d[item] >= 2:
        popular_tag_list.append(item)

In [ ]:
#finding frequency of tags mentioned in dataset
tag_count_d = {}
for item in name_tag_dict.values():
    for tag in item:
        if tag in tag_count_d:
            tag_count_d[tag] += 1 
        else:
            tag_count_d[tag] = 1          

In [ ]:
#creating newtork graph

In [ ]:
G = nx.Graph()

In [ ]:
pd.options.display.max_colwidth = 50000

In [ ]:
new_dict = {'col_1': list(), 'col_2': list(), 'col_3': list()}
    
for key in name_tag_dict.keys():
    if '?' not in key:
        temp_tags = name_tag_dict[key]
        for person in name_tag_dict.keys():
            if person != key:
                if '?' not in person:
                    temp_tags_2 = name_tag_dict[person]
                    for item in temp_tags_2:
                        if item in temp_tags and item in popular_tag_list:
                            new_dict['col_1'].append(key)
                            new_dict['col_2'].append(person)
                            new_dict['col_3'].append(item)


In [ ]:
new_df = pd.DataFrame.from_dict(new_dict)

t = new_df.drop_duplicates()
t

In [ ]:
G = nx.from_pandas_edgelist(df = t, source = 'col_1', target = 'col_2', edge_attr = 'col_3')


In [ ]:
new_dict = {}
for index, row in t.iterrows():
    
    new_dict[(row.col_1, row.col_2)] = row.col_3

In [ ]:
nx.set_edge_attributes(G, new_dict, 'Tag Relationship')


In [ ]:
get_attr = nx.get_edge_attributes(G, 'Tag Relationship')

In [ ]:
node_trace = go.Scattergl(x = [], y = [], text = [], mode = 'markers',
                    marker = go.Marker(
                    showscale = True,
                    reversescale = True,
                    color = 'maroon',
                    size = 15,
                    line = {'width': 2}))

In [ ]:
pos = nx.spring_layout(G)
dmin=1
ncenter=0
for n in pos:
    x,y=pos[n]
    d=(x-0.5)**2+(y-0.5)**2
    if d<dmin:
        ncenter=n
        dmin=d

p=nx.single_source_shortest_path_length(G,ncenter)

for node in G.nodes():
    G.node[node]['pos']= pos[node]



In [ ]:
node_list = []
for node in G.nodes():
    x, y = G.node[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_list.append(node)
    node_trace['customdata'] = tuple(node_list)


In [ ]:
   
edge_trace = go.Scattergl(x=[], y=[], line={'width':1.0,'color': 'purple'}, mode='lines')


In [ ]:
get_attr = nx.get_edge_attributes(G, 'Tag Relationship')
edge_list = []
for edge in G.edges():
    x0, y0 = G.node[edge[0]]['pos']
    x1, y1 = G.node[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])
    edge_list.append(get_attr[edge])
    edge_trace['customdata'] = tuple(edge_list)
    
        


In [ ]:
groups = list(set(t.col_3))


In [ ]:
fig = go.Figure(data = go.Data([edge_trace, node_trace]),
             layout = go.Layout(width=800,
    height=900,
    autosize=False,
                 title = 'Jane Addams and her Chicago Network',
                 titlefont = {'size':16},
                 showlegend = True,
                 margin = {'b':20, 'l':5, 'r':5, 't':40},
                 xaxis = go.XAxis(title = 'xaxis',showgrid = False, 
                               zeroline = False, 
                               showticklabels = False),
                 yaxis = go.YAxis(title = 'yaxis', showgrid = False, 
                               zeroline = False, 
                               showticklabels = False)))





py.iplot(fig, filename = 'networkx')



